In [2]:
import os
import pandas as pd
import sys
PROJECT_ROOT = os.path.abspath("..")
sys.path.insert(0, PROJECT_ROOT)
from script.csvs_to_sql import detect_delimiter
CSV_PATH = "../data_acidentes_prf/"
os.getcwd()

'/home/wallace/projetos/python/projects/data_prf_gov/project_data_prf_gov/notebooks'

In [3]:
years = [ano for ano in range(2007,2026)]
years

[2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020,
 2021,
 2022,
 2023,
 2024,
 2025]

In [ ]:
def files_per_year(year:str) -> list[str]:
    files_per_year = []
    files = os.listdir(CSV_PATH)
    for file in files:
        if year in file:
            files_per_year.append(file)
    return files_per_year

def concat_df(files:list[str]) -> pd.DataFrame:
    dfs = [
    pd.read_csv(
        f"{CSV_PATH}{file}",
        encoding="latin1",
        sep=detect_delimiter(f"{CSV_PATH}{file}")
    )
    for file in files
    ]

    df = pd.concat(dfs, ignore_index=True)
    df = df.drop_duplicates()
    return df

files = files_per_year("2007")
df = concat_df(files)

/tmp/ipykernel_43484/4070234430.py:11: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(
/tmp/ipykernel_43484/4070234430.py:11: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(
